# Spanish Word Embeddings

# This module implements word vectors and their similarity look-ups.

we use gesim package to perform a wormbeddings (word2vec) transformation in order we can compare the users text and company textsuch that a match can be done between them 

In [1]:
import numpy as np
import pandas as pd
import math
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

In [1]:
#installing gensim
#import sys
#!conda install -c conda-forge gensim --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda

  added / updated specs:
    - gensim


The following packages will be UPDATED:

  gensim             pkgs/main::gensim-3.4.0-py36hfa6e2cd_0 --> conda-forge::gensim-3.7.3-py36h6538335_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


# LOADING THE WORD2VEC MODEL

loading a pre-trained spanish word-embedding, with a vector dim of 300 and 1000000 vectors -> for more infortion see the web site : https://github.com/dccuchile/spanish-word-embeddings

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'fasttext-sbwc.3.6.e20.vec'
cantidad = 1000000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [19]:
#example 
vector = wordvectors.get_vector('casa')
print(vector)

[ 1.2100e-01 -1.5552e-01 -3.5114e-02 -1.7998e-01 -1.6266e-01 -1.9180e-01
 -2.5319e-01  1.0108e-01 -1.1969e-01  7.4725e-03 -1.0380e-01  9.8222e-02
  2.8586e-01 -1.2828e-01  2.3204e-01 -1.3754e-03 -1.3301e-01  2.7393e-01
 -3.4402e-02  1.2702e-01 -3.1093e-01 -5.1120e-01 -9.3321e-02 -3.6300e-02
 -6.9176e-02 -1.6083e-01  5.1955e-02 -2.1072e-01 -2.6308e-01 -4.8167e-01
 -4.2471e-01  3.2596e-02 -2.4753e-01  3.7219e-01  3.9991e-02 -4.8988e-02
 -1.2377e-02  1.7868e-01  2.8374e-01  3.5436e-01 -3.9200e-01 -1.3087e-01
 -6.1445e-02  2.0615e-01  4.8538e-01  8.1507e-02 -3.8315e-03  1.1456e-01
  1.9641e-01 -3.1384e-02  1.7718e-01  1.5452e-01  1.2188e-01  1.4018e-01
 -2.2728e-02 -2.0315e-01  4.3390e-02 -3.3416e-01  3.3122e-01 -2.0819e-02
 -7.7117e-02 -4.3856e-02  6.5116e-02  9.3585e-02 -9.1272e-02  1.1774e-01
  1.1442e-01  3.1598e-01  2.7401e-01  4.8852e-03 -1.2554e-01 -7.8936e-02
 -2.8074e-01 -1.9653e-01  8.7350e-02 -2.1922e-01  2.2284e-01 -4.9907e-02
 -5.5925e-01 -3.1795e-01  1.6967e-01  6.9432e-02 -3

#  LOADING THE USERS & COMPANY DATA FOR THE MATCH

In [4]:
# create a pandas dataframe
data_users = pd.read_csv(file1, error_bad_lines=False) 
data_companies = pd.read_csv(file2, error_bad_lines=False) 

In [5]:
data_users = data_users[['user_id','completo']]
len(data_users)

9893

In [6]:
demogra_companies = data_companies[['id','name']]
data_companies2 = data_companies[['name','completo_company']]
len(data_companies2)

10

# OBTAING THE EMBEDDINGS FOR EACH TEXT IN USERS AND COMPANIES

1) Split the text (user and company) to each word 

2) obtain the word embedding for each word

3) contruct a mean embedding representation for the complete text (user and company)

4) compute the distance between each user mean embedding to each company mean embedding -> the minor distance stablish the macth!

In [7]:
def sentence_to_avg(sentence):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 300-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (300,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = sentence.lower().split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(300)
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    for w in words:
        try:
            wv = wordvectors.get_vector(w)
        except:
            continue
        total += 1
        avg = avg + wv
        
    if total != 0:
        avg = avg/total
    #print(total)
    ### END CODE HERE ###
    
    return avg

In [25]:
# EJEMPLO DE USO

# se usa el modelo para obtener el embedding para cada palabra en el texto
vector = sentence_to_avg('la casa esta limpia y es de color azul en la casa hay una niña que canta') #la casa esta limpia y es de color azul en la casa hay una niña que canta
len(vector)

300

# company word-embeddings

In [54]:
# loop over the company text and return the vector
data_companies2['vector_c'] = data_companies2.completo_company.apply(lambda x : sentence_to_avg(x))

display(data_companies2)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


name  \
0  Alianza Team®                                         
1  Sura                                                  
2  Interacpedia                                          
3  Bancolombia                                           
4  Emtelco                                               
5  Konecta                                               
6  Satrack INC de Colombia Servisat                      
7  Auteco Mobility                                       
8  Auteco SAS                                            
9  Comfama - Caja de Compensación Familia de Antioquia   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [59]:
#vc = data_companies2['vector_c'].loc[data_companies2['name']=='Konecta']
#vc

In [52]:
#pd.set_option('display.max_colwidth', -1)
#print(vc)

# user word-embeddings

In [8]:
# loop over the company text and return the vector
data_users['vector_u'] = data_users.completo.apply(lambda x : sentence_to_avg(x))

display(data_users).head(10)

,user_id,completo,vector_u
0,2146,es visioes office promodeles gsuites amples ca...,"[0.11858514252041157, -0.14668953928533787, 0...."
1,29075,ciencias sociales hum objetivos surgen aporte ...,"[0.11510586094722719, -0.21699107455052222, 0...."
2,36281,ciencias sociales hum innovacion aprendizaje ...,"[0.1125009817231832, -0.1826688452230381, -0.0..."
3,5245,ciencias aplicadas,"[0.24338049814105034, -0.33135999739170074, 0...."
4,26016,ciencias sociales hum httpwwwalmaceneslamedian...,"[0.10729864580048756, -0.1783078551165421, -0...."
5,1477,es honestidad estudiar administracion,"[0.16358525399118662, -0.25315525382757187, 0...."
6,5373,ciencias aplicadas,"[0.24338049814105034, -0.33135999739170074, 0...."
7,3889,ciencias sociales hum objetivo impactar centro...,"[0.16171668923925608, -0.15790062450105324, 0...."
8,33418,ciencias aplicadas referred aily gomez pardo,"[0.1647584984699885, -0.33821166555086773, -0...."
9,6171,ciencias sociales hum,"[0.42536666989326477, -0.09722333153088887, 0...."


AttributeError: 'NoneType' object has no attribute 'head'

In [60]:
#vu = data_users['vector_u'].loc[data_users['user_id']==12]
#vu

# MATCH BETWEEN USER AND COMPANY

The match is computed by the use of the euclidean distance between the embedding vectors. So that the mininmun distance between them indicates the match.

In [28]:
def euliden_distace(x1,x2):
    dist = math.sqrt((x1-x2)**2)
    return(dist)

def cosine_distace(x1,x2):
    dist_co = dot(x1, x2)/(norm(x1)*norm(x2)) 
    return(dist_co)

In [17]:
cosine_similarity([[1, 0, -1, 5]], [[-1,-1, 0, 4]])

array([[0.8618575]])

In [61]:
M = np.zeros((len(data_companies),len(data_users)), dtype=float)
M.shape

(10, 9893)

In [62]:
df = pd.DataFrame(M)
df['company'] = data_companies2['name']
df.columns = data_users.user_id.tolist() + ['company']
#display(df)

In [63]:
for i in range(0,len(data_users)):   #
    for j in range(0,len(data_companies2)):  #
        #RESHAPE THE VECTORS
        V1 = data_users.vector_u[i].reshape(1, -1)
        V2 = data_companies2.vector_c[j].reshape(1, -1) 
        # COMPUTE THE COSINE SIMILARITY
        df.iloc[j,i] = np.asscalar(cosine_similarity(V1,V2))
             

In [33]:
display(df) 

,2146,29075,36281,5245,26016,1477,5373,3889,33418,6171,...,35387,9420,36816,39250,3024,37812,29632,6872,857,company
0,0.878584,0.938633,0.967355,0.457251,0.952744,0.733991,0.457251,0.824180,0.697133,0.529107,...,0.957251,0.529107,0.972052,0.457251,0.424357,0.946139,0.457251,0.794337,0.729203,Alianza Team®
1,0.886940,0.966595,0.949782,0.473615,0.940629,0.739897,0.473615,0.832972,0.675222,0.539914,...,0.953752,0.539914,0.942174,0.473615,0.418772,0.921869,0.473615,0.813926,0.704713,Sura
2,0.893913,0.935355,0.958812,0.472036,0.940963,0.733680,0.472036,0.829114,0.717977,0.542855,...,0.943990,0.542855,0.958958,0.472036,0.431612,0.928622,0.472036,0.789707,0.739865,Interacpedia
3,0.880526,0.949209,0.954949,0.479264,0.941080,0.734261,0.479264,0.825277,0.692602,0.539066,...,0.950128,0.539066,0.955934,0.479264,0.419648,0.929506,0.479264,0.797751,0.732927,Bancolombia
4,0.893923,0.946668,0.967365,0.462540,0.953608,0.744822,0.462540,0.828740,0.701007,0.536725,...,0.955875,0.536725,0.966684,0.462540,0.418605,0.944450,0.462540,0.804279,0.736891,Emtelco
5,0.880417,0.940216,0.970279,0.459972,0.959604,0.740949,0.459972,0.839143,0.704153,0.541286,...,0.961820,0.541286,0.968633,0.459972,0.427858,0.943421,0.459972,0.800869,0.731936,Konecta
6,0.902745,0.948639,0.960328,0.480372,0.942204,0.731355,0.480372,0.830822,0.705514,0.537385,...,0.951933,0.537385,0.960557,0.480372,0.411325,0.933497,0.480372,0.826873,0.747986,Satrack INC de Colombia Servisat
7,0.888517,0.952733,0.965577,0.457288,0.951350,0.739853,0.457288,0.815346,0.671736,0.516751,...,0.951524,0.516751,0.960353,0.457288,0.413461,0.939017,0.457288,0.816756,0.734269,Auteco Mobility
8,0.858670,0.931936,0.965443,0.456862,0.947359,0.728565,0.456862,0.829863,0.692818,0.530183,...,0.947899,0.530183,0.963064,0.456862,0.436118,0.940452,0.456862,0.795890,0.722822,Auteco SAS
9,0.885751,0.955466,0.964410,0.478404,0.950270,0.733370,0.478404,0.842777,0.703320,0.558482,...,0.956260,0.558482,0.952391,0.478404,0.439123,0.937494,0.478404,0.813833,0.705861,Comfama - Caja de Compensación Familia de Anti...


In [74]:
#df[[12,'company']]

In [34]:
#crate an empy data frame
df_match = pd.DataFrame(columns=['user_id','company'], index = data_users['user_id'].tolist())

In [35]:
#case 1: match for the minimum distance -> the most similarity vectors
for col in (df.columns[:-1]):
    maximo = df[col].max()  
    df_match.loc[col,'user_id'] = col
    #df_match.loc[col, 'score'] = maximo
    df_match.loc[col,'company'] = (df[df[col] == maximo]['company'].tolist()[0])
    #print(col,maximo,(df[df[col] == maximo]['company'].tolist()[0]))

In [36]:
df_match.tail(10)

,user_id,company
3618,3618,Konecta
35387,35387,Konecta
9420,9420,Comfama - Caja de Compensación Familia de Anti...
36816,36816,Alianza Team®
39250,39250,Satrack INC de Colombia Servisat
3024,3024,Comfama - Caja de Compensación Familia de Anti...
37812,37812,Alianza Team®
29632,29632,Satrack INC de Colombia Servisat
6872,6872,Satrack INC de Colombia Servisat
857,857,Satrack INC de Colombia Servisat


In [37]:
#case 2: macth using the max and the mean
for col in (df.columns[:-1]):
    maximo = df[col].max()  
    promedio = df[col].mean()  
    df_match.loc[col,'user_id'] = col
    #df_match.loc[col, 'score'] = maximo
    #print(col,df[(df[col]<=maximo)&(df[col] > promedio)]['company'].tolist())
    df_match.loc[col,'company'] = str(df[(df[col]<=maximo)&(df[col] >= promedio)].sort_values(by=col,ascending = False)['company'].tolist())
    #print(col,df[(df[col]<=maximo)&(df[col] > promedio)]['company'].tolist())

In [38]:
df_match.tail(10)

,user_id,company
3618,3618,"['Konecta', 'Emtelco', 'Bancolombia', 'Satrack..."
35387,35387,"['Konecta', 'Alianza Team®', 'Comfama - Caja d..."
9420,9420,['Comfama - Caja de Compensación Familia de An...
36816,36816,"['Alianza Team®', 'Konecta', 'Emtelco', 'Autec..."
39250,39250,"['Satrack INC de Colombia Servisat', 'Bancolom..."
3024,3024,['Comfama - Caja de Compensación Familia de An...
37812,37812,"['Alianza Team®', 'Emtelco', 'Konecta', 'Autec..."
29632,29632,"['Satrack INC de Colombia Servisat', 'Bancolom..."
6872,6872,"['Satrack INC de Colombia Servisat', 'Auteco M..."
857,857,"['Satrack INC de Colombia Servisat', 'Interacp..."


# WRITING THE RESULTS IN THE DATA BASE

create the data in SQL server. Ejecute this query in SQL:

CREATE TABLE match(id INT identity primary key,

[user_id] VARCHAR(255)

,[company] text)

In [39]:
# save the results to the data base
import pyodbc 
import platform 

driver = ''

if platform.system() == 'Windows':
    driver = '{SQL Server}'
else:
    driver = 'ODBC Driver 17 for SQL Server'
            
conection = pyodbc.connect(
    
cursor = conection.cursor()


for index,row in df_match.iterrows():
    cursor.execute("INSERT INTO dbo.match_all_user([user_id],[company])values (?,?)", row['user_id'],row['company']) 
    
conection.commit()
cursor.close()
conection.close()



# NATURAL SEGMENTATION FOR THE USERS

A k-means++ will be used for the natural segmentation

1) the user vectors are used to training the kmeans

In [61]:
#assing each user vector to a new array
Z = []
for value in data_users['vector_u']:
    #for v in value
    Z.append(value)
    
Z = np.array(Z)   
len(Z)

9893

In [67]:
#set the clusters number to 5 and use the kmeans++ to get the clusters labels for each user
true_k = 5
kmeans = KMeans(n_clusters=true_k, init='k-means++', max_iter=100)
kmeans.fit(Z)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [68]:
center_coordinates = kmeans.cluster_centers_
center_coordinates

array([[ 0.19091923, -0.28408007, -0.05610869, ...,  0.2559294 ,
         0.10602907, -0.05433201],
       [ 0.41628873, -0.10700022,  0.03573173, ...,  0.21526588,
         0.10863725, -0.10114703],
       [ 0.24040743, -0.33133531,  0.05424821, ...,  0.18369881,
         0.07638359,  0.10205099],
       [ 0.11873365, -0.19819873, -0.01358487, ...,  0.12938653,
         0.05383716, -0.11853204],
       [ 0.30929508, -0.1187195 ,  0.08360709, ...,  0.57242842,
        -0.10978068,  0.34996979]])

In [69]:
cluters_labels = kmeans.labels_
cluters_labels

array([3, 3, 3, ..., 2, 3, 0])

In [72]:
#assigning the cluster to each user
data_users['cluster'] = cluters_labels
data_users.head(10)

,user_id,completo,vector_u,cluster
0,2146,es visioes office promodeles gsuites amples ca...,"[0.11858514252041157, -0.14668953928533787, 0....",3
1,29075,ciencias sociales hum objetivos surgen aporte ...,"[0.11510586094722719, -0.21699107455052222, 0....",3
2,36281,ciencias sociales hum innovacion aprendizaje ...,"[0.1125009817231832, -0.1826688452230381, -0.0...",3
3,5245,ciencias aplicadas,"[0.24338049814105034, -0.33135999739170074, 0....",2
4,26016,ciencias sociales hum httpwwwalmaceneslamedian...,"[0.10729864580048756, -0.1783078551165421, -0....",3
5,1477,es honestidad estudiar administracion,"[0.16358525399118662, -0.25315525382757187, 0....",3
6,5373,ciencias aplicadas,"[0.24338049814105034, -0.33135999739170074, 0....",2
7,3889,ciencias sociales hum objetivo impactar centro...,"[0.16171668923925608, -0.15790062450105324, 0....",3
8,33418,ciencias aplicadas referred aily gomez pardo,"[0.1647584984699885, -0.33821166555086773, -0....",0
9,6171,ciencias sociales hum,"[0.42536666989326477, -0.09722333153088887, 0....",1


In [80]:
#example 
vector = wordvectors.get_vector('casa')
wordvectors.similar_by_vector(vector, topn=10, restrict_vocab=None)

[('casa', 0.9999999403953552),
 ('mansión', 0.6957781910896301),
 ('casita', 0.6807630062103271),
 ('solariega', 0.6178374290466309),
 ('casona', 0.6127971410751343),
 ('casas', 0.6014595031738281),
 ('apartamento', 0.5869998335838318),
 ('caserón', 0.5847396850585938),
 ('finca', 0.579517662525177),
 ('habitación', 0.5749081373214722)]

In [83]:
vetor_c1 = center_coordinates[0]
wordvectors.similar_by_vector(vetor_c1, topn=20, restrict_vocab=None)

[('productsupplycenter', 0.7424926161766052),
 ('contactoffice', 0.7344679832458496),
 ('softwarelivre', 0.733843982219696),
 ('googleartproject', 0.7310038805007935),
 ('currentversion', 0.7294415235519409),
 ('softmotion', 0.7258033752441406),
 ('pocketstudio', 0.7245789766311646),
 ('xmlwriter', 0.7213563919067383),
 ('videodesktop', 0.7201676368713379),
 ('windowsce', 0.7185452580451965),
 ('ajaxwrite', 0.7141598463058472),
 ('expressionengine', 0.7135368585586548),
 ('pictureproject', 0.7131321430206299),
 ('friendware', 0.7111128568649292),
 ('phonetools', 0.7110444903373718),
 ('freeculture', 0.7088040709495544),
 ('activepython', 0.7071545124053955),
 ('pipeworks', 0.7067310810089111),
 ('signalexpress', 0.7067011594772339),
 ('windowsmedia', 0.7053967714309692)]

In [84]:
vetor_c2 = center_coordinates[1]
wordvectors.similar_by_vector(vetor_c2, topn=20, restrict_vocab=None)

[('ciencias', 0.7409349083900452),
 ('hum', 0.6920563578605652),
 ('humanidades', 0.6723480224609375),
 ('agrociencias', 0.6507672667503357),
 ('sociales', 0.6230109930038452),
 ('sociología', 0.610287070274353),
 ('sociologías', 0.601740300655365),
 ('ciencas', 0.6001553535461426),
 ('sciencias', 0.5863096714019775),
 ('neurosociología', 0.5700617432594299),
 ('microsociología', 0.565043032169342),
 ('agronomía', 0.5641521215438843),
 ('archivología', 0.5630977153778076),
 ('ciberantropología', 0.5627626180648804),
 ('biomédicas', 0.5581976175308228),
 ('sociomédicas', 0.5578142404556274),
 ('cienca', 0.5577612519264221),
 ('filologías', 0.5573064088821411),
 ('etnosociología', 0.5572402477264404),
 ('ciencies', 0.5532931685447693)]

In [86]:
vetor_c3 = center_coordinates[2]
wordvectors.similar_by_vector(vetor_c3, topn=20, restrict_vocab=None)

[('ciencias', 0.871685266494751),
 ('aplicadas', 0.797925591468811),
 ('humanidades', 0.7147239446640015),
 ('matemáticas', 0.6783336400985718),
 ('ciencas', 0.6619595885276794),
 ('agronomía', 0.6453644633293152),
 ('ingenierías', 0.6453312635421753),
 ('sociología', 0.6423039436340332),
 ('artes', 0.6363753080368042),
 ('facultad', 0.6360808610916138),
 ('catedráticas', 0.6360771656036377),
 ('doctoradas', 0.6358554363250732),
 ('geociencias', 0.6273008584976196),
 ('agrociencias', 0.6268049478530884),
 ('biología', 0.6231940388679504),
 ('fisicomatemáticas', 0.621130645275116),
 ('sociologías', 0.61934494972229),
 ('ingeniería', 0.6192710399627686),
 ('biologías', 0.6166793704032898),
 ('filologías', 0.6108869910240173)]

In [87]:
vetor_c4 = center_coordinates[3]
wordvectors.similar_by_vector(vetor_c4, topn=20, restrict_vocab=None)

[('mostrarporpantalla', 0.6637923717498779),
 ('lenguaventura', 0.637901782989502),
 ('visiontutor', 0.6279415488243103),
 ('travelbubble', 0.6253179311752319),
 ('desafiointel', 0.6238630414009094),
 ('anuncioscasasgratis', 0.6226706504821777),
 ('googleartproject', 0.6221367120742798),
 ('personalizaciones', 0.6218532919883728),
 ('que', 0.6177679300308228),
 ('adultfriendfinder', 0.6151204109191895),
 ('busquedagoogle', 0.6137664318084717),
 ('citysiesta', 0.6132045388221741),
 ('secretariaplus', 0.6089305281639099),
 ('materialdelectura', 0.6087946891784668),
 ('personalizacion', 0.6087530851364136),
 ('paltalkscene', 0.6077431440353394),
 ('regcure', 0.6070301532745361),
 ('visualidades', 0.6058428287506104),
 ('lcdstudio', 0.605327844619751),
 ('cocinaconencanto', 0.6048035621643066)]

In [88]:
vetor_c5 = center_coordinates[4]
wordvectors.similar_by_vector(vetor_c5, topn=20, restrict_vocab=None)

[('artes', 0.9999909996986389),
 ('bellas', 0.793185830116272),
 ('escénicas', 0.7271032333374023),
 ('plásticas', 0.6946129202842712),
 ('arte', 0.6847355365753174),
 ('marciales', 0.6564899682998657),
 ('ciencias', 0.6314058899879456),
 ('humanidades', 0.6032871007919312),
 ('academia', 0.596428394317627),
 ('artísticas', 0.5800008773803711),
 ('inba', 0.5761932730674744),
 ('decorativas', 0.5724005699157715),
 ('cinematografía', 0.5687955617904663),
 ('conservatorio', 0.5608032941818237),
 ('precinematografía', 0.5577881336212158),
 ('artísticos', 0.5571895837783813),
 ('danza', 0.5554196238517761),
 ('visuales', 0.5464198589324951),
 ('belas', 0.5457320213317871),
 ('escéncias', 0.5439279079437256)]